# Program 2

- Train one or more detection models via transfer learning.
- Evaluate hyperparameter choices and how they affect the learning.
- Use the validation set to validate parameter choices.
- Choose the best model, i.e., best classifier + best hyper-parameter choices.

In [2]:
import torch

In [3]:
torch.cuda.empty_cache()

In [4]:
#!pip install -U ultralytics "ray[tune]"
#!pip install wandb

In [5]:
!pip install -U ultralytics

Defaulting to user installation because normal site-packages is not writeable


In [6]:
#!pip uninstall wandb -y

In [7]:
from PIL import Image
import cv2
from ultralytics import YOLO

In [8]:
torch.cuda.is_available()

True

In [8]:
#!yolo version
#import os
#cwd=os.getcwd()
#print(cwd)

#os.chdir("/WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2")
#cwd=os.getcwd()
#print(cwd)

In [9]:
#!yolo version

/bin/bash: yolo: command not found


In [10]:
#!ultralytics --version

/bin/bash: ultralytics: command not found


## Preprocessing Data

- YOLO works best on square image sizing
- It can do rectangular images, but both height width need to be multiples of 32
- Given dataset has images of size 960 x 540 -> 540 is not a multiple of 32
- So our first approach will be to square the images
- According this this source a non-multiple of 32 should be fine? (https://github.com/ultralytics/ultralytics/issues/3815) Lets try it first

- Created a custom_data folder that contains symbolic links to the image folders, not the whole dataset
- This is because we need to have a labels folder with individual label files for each image and by default, we could not edit the file structure of the given dataset

- YOLO requires a dataset formatted in a particular way
- each image should have it's own label.txt with the format: {class_id}{x_center}{y_center}{width}{height}
- label.txt name should match the image file name
- x_center, y_center, width, and height should be normalized (value between 0-1)

In [11]:
#the following function does these things, for the specified dataset
def create_label_files(dataset):
    label_file_path = "/WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/data/" + dataset + "/labels.txt"
    label_dir_path = "/WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/" + dataset + "/labels"
    image_dir_path = "/WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/" + dataset + "/images"

    #read label.txt file
    with open(label_file_path, "r") as file:
        lines = file.readlines()

    #dictionary to store labels for each image
    image_labels = {}
    
    #line format: {image_id}{class_id}{x_center}{y_center}{width}{height}
    for line in lines:
        split = line.strip().split()
        image_id, class_id, x_center, y_center, box_width, box_height = split

        #now that we have the components of the line, we store them in the image_labels dicitonary
        #key is image_id, value are label components
        if image_id not in image_labels:
            image_labels[image_id] = []
        image_labels[image_id].append((class_id, x_center, y_center, box_width, box_height))

    #now that we have the labels in the dictionary, we need to process and normalize the values
    for image_id, labels in image_labels.items():
        #image file names have leading 0's 
        image_file_name = f"{int(image_id):05d}.jpeg"
        image_file_path = image_dir_path + "/" + image_file_name

        #gather size of image, used for normalizing values
        with Image.open(image_file_path) as image:
            image_width, image_height = image.size

        #normalize label components
        normalized_labels = []
        for label in labels:
            class_id, x_center, y_center, width, height = label

            x_center_normalized = float(x_center) / image_width
            y_center_normalized = float(y_center) / image_height
            width_normalized = float(width) / image_width
            height_normalized = float(height) / image_height

            #create label values with normalized values
            #YOLO documentation shows normalized values up to 6 decimal places
            normalized_label = f"{class_id} {x_center_normalized:.6f} {y_center_normalized:.6f} {width_normalized:.6f} {height_normalized:.6f}"
            normalized_labels.append(normalized_label)


        new_label_file_path = label_dir_path + "/" + f"{int(image_id):05d}.txt"
        with open(new_label_file_path, "w") as output:
            output.write("\n".join(normalized_labels))

In [12]:
#create the label files in custom_data
#only need to run this once
create_label_files("val")
create_label_files("train")

## Training with YOLO
- load the pre-trained yolo model
- re-train using the custom dataset

In [ ]:
# build and load YOLO
model = YOLO("yolo11n.pt")
#model = YOLO("yolo11x.pt")
model.cuda()
results = model.train(data="dataset.yaml", epochs=100, imgsz=(960, 540), lr0=0.001, rect=True, save=True, device=0, optimizer="auto")

#resume training model
#model = YOLO("/WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/runs/detect/train20/weights/last.pt")
#model.cuda()
#results = model.train(data="dataset.yaml", epochs=100, imgsz=(960, 540), rect=True, save=True, device=0, lr0=0.001, optimizer="auto", resume=True)

Ultralytics 8.3.78 🚀 Python-3.9.21 torch-2.6.0+cu124 CUDA:0 (Tesla V100-PCIE-32GB, 32494MiB)
engine/trainer: task=detect, mode=train, model=/WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/runs/detect/train20/weights/last.pt, data=dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=(960, 540), save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train20, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=True, cos_lr=False, close_mosaic=10, resume=/WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/runs/detect/train20/weights/last.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=Non

train: Scanning /WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/train/labels.cache... 14000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 14000/14000 [00:00<?, ?it/s]

train: WARNING ⚠️ /WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/train/images/00288.jpeg: 1 duplicate labels removed
train: WARNING ⚠️ /WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/train/images/00491.jpeg: 1 duplicate labels removed
train: WARNING ⚠️ /WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/train/images/00787.jpeg: 1 duplicate labels removed
train: WARNING ⚠️ /WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/train/images/00802.jpeg: 2 duplicate labels removed
train: WARNING ⚠️ /WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/train/images/01505.jpeg: 1 duplicate labels removed
train: WARNING ⚠️ /WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/train/images/02366.jpeg: 1 duplicate labels removed
train: WARNING ⚠️ /WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/train/images/02431.jpeg: 1 duplicate labels removed
train: WARNING ⚠️ /WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/

WARNING ⚠️ 'rect=True' is incompatible with DataLoader shuffle, setting shuffle=False


/WAVE/users2/unix/emitchell/.local/lib/python3.9/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
val: Scanning /WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/val/labels.cache... 2000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2000/2000 [00:00<?, ?it/s]

...

                   all       2000       9061      0.602      0.605      0.636      0.414

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     31/100      2.83G      1.289     0.6839      1.415         73        960: 100%|██████████| 875/875 [03:44<00:00,  3.90it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:15<00:00,  3.96it/s]

                   all       2000       9061      0.553      0.674      0.638      0.418



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     32/100      2.74G      1.285     0.6799      1.419         63        960:  39%|███▉      | 342/875 [01:27<02:25,  3.67it/s]

## Evaluate YOLO Model

In [9]:
#evaluate trained model

#metrics = YOLO("/WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/runs/detect/train20/weights/best.pt")

metrics = results.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category



#results = model.val()
#print(metrics.box.map)  # mAP50-95
#print(metrics.box.map50)  # mAP50
#print(metrics.box.map75)  # mAP75
#print(metrics.box.maps)  # list of mAP50-95 for each category

Ultralytics 8.3.78 🚀 Python-3.9.21 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce GTX 1080, 8107MiB)
YOLO11n summary (fused): 100 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/val/labels.cache... 2000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2000/2000 [00:00<?, ?it/s]

val: WARNING ⚠️ /WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/val/images/00425.jpeg: 1 duplicate labels removed
val: WARNING ⚠️ /WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/val/images/00954.jpeg: 1 duplicate labels removed
val: WARNING ⚠️ /WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/val/images/01642.jpeg: 1 duplicate labels removed
val: WARNING ⚠️ /WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/val/images/01755.jpeg: 1 duplicate labels removed



/WAVE/users2/unix/emitchell/.local/lib/python3.9/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 125/125 [00:19<00:00,  6.41it/s]


                   all       2000       9061      0.588      0.629      0.641      0.425
                   car       1978       8781      0.807      0.877      0.901      0.575
          medium truck        154        163      0.494      0.564      0.499      0.343
           large truck        110        117      0.462      0.444      0.522      0.357
Speed: 0.3ms preprocess, 4.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val4


array([    0.42519,     0.57476,     0.34342,     0.35738])

## Inference results

In [9]:

#model = YOLO("/WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/runs/detect/train20/weights/best.pt")

test = results.predict(source="/WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/test/images", save_txt=True);




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/2000 /WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/test/images/00001.jpeg: 544x960 4 cars, 36.3ms
image 2/2000 /WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/test/images/00002.jpeg: 544x960 2 cars, 9.7ms
image 3/2000 /WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/test/images/00003.jpeg: 544x960 11 cars, 7.7ms
image 4/2000 /WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/custom_data/test/images/

## We need to append the confidence scores to the end of each results file

In [ ]:
# Assuming you have results from model.predict()

#test[0].boxes.cls
#print("1: ", test[0].boxes.conf)
#print("2: ", test[1].boxes.conf)

#if test.boxes is not None:  # Check if any boxes are detected
#    for box in test.boxes:
#        print(f"Coordinates: {test.xyxy}, Confidence: {test.conf.item():.2f}")

In [11]:
#format output files to original format
#currently we have an output file for every test image (2,000) total
#need to create a singular output file that puts converts the file name to a record with image id (00001.jpeg -> image_id: 1)

import os

def format_output_files(predictions_path):
    #gather prediction files
    output_path = "/WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/predictions.txt"
    predictions_files = sorted([file for file in os.listdir(predictions_path) if file.endswith(".txt")])

    format_lines = []

    #if we hae a filename like 00001.txt
    #we need to convert this name to image_id
    for filename in predictions_files:
        image_id = os.path.splitext(filename)[0]
        image_id= int(image_id)

        file_path = os.path.join(predictions_path, filename)

        with open(file_path, "r") as file:
            lines = file.readlines()

        #before stripping the lines, we have to append the confidence score
        try:
            conf_tensor = test[image_id - 1].boxes.conf
            conf_list = conf_tensor.cpu().numpy().tolist()
        except Exception as e:
            print(f"Error retrieving confidence scores for image {image_id}: {e}")
            continue

        if len(lines) != len(conf_list):
            print(f"Warning: For image {image_id}, number of predictions ({len(lines)}) does not match number of confidence scores ({len(conf_list)})")
        
        for i, line in enumerate(lines):
            line = line.strip()
            if not line:
                continue
            try:
                conf_score = conf_list[i]
            except IndexError:
                conf_score = 0.0
        
        #for line in lines:
        #    line = line.strip()
        #    if not line:
        #        continue

    
            #we reformat the lines
            #format_line = f"{image_id} {line}"
            #format_lines.append(format_line)

            format_line = f"{image_id} {line} {conf_score:.6f}"
            format_lines.append(format_line)


    #write it to the output file
    with open(output_path, "w") as output_file:
        output_file.write("\n".join(format_lines))
        
 
#path is the directory of all of the predictions
#final output file at path: "/WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/predictions.txt"
format_output_files("/WAVE/archive/users/emitchell/CSEN342W25/CSEN342P2/runs/detect/predict6/labels/")
